In [19]:
# This was used to check for missing files. Some of the files listed in samples.csv had broken download links.
# Since the data was too big to include, this is omitted using the check variable and included for the sake of completeness.
import os
import pandas as pd

def checkformissing():
    filelist = []

    for file in os.listdir("D:\ML\data\FWM"):
        if file.endswith(".bam"):
            filelist.append(file[0:3])

    return filelist

In [9]:
import pandas as pd
import numpy as np

donor_df = pd.read_csv("donorinfo.csv")

donor_df['Treatment'] = np.nan
donor_df['cerad'].replace(to_replace=[0,1], value=0, inplace=True)
donor_df['cerad'].replace(to_replace=[2,3], value=1, inplace=True)
donor_df['dsm_iv_clinical_diagnosis'].replace(to_replace=["Alzheimer's Disease Type", 'Multiple Etiologies', 'Other Medical', 'Other or Unknown Cause', 'Vascular'], value=0, inplace=True)
donor_df['dsm_iv_clinical_diagnosis'].replace(to_replace= 'No Dementia', value=1, inplace=True)
donor_df['nincds_arda_diagnosis'].replace(to_replace=["Possible Alzheimer'S Disease", "Probable Alzheimer'S Disease"], value=1, inplace=True)
donor_df['nincds_arda_diagnosis'].replace(to_replace=["No Dementia", "Dementia, Type Unknown"], value=0, inplace=True)
donor_df['act_demented'].replace(to_replace="Dementia", value=1, inplace=True)
donor_df['act_demented'].replace(to_replace="No Dementia", value=0, inplace=True)
donor_df['braak'].replace(to_replace=[1,2,3,4,5,6], value=1, inplace=True)
donor_df['dsm_iv_clinical_diagnosis'].sum()

for i, row in donor_df.iterrows():
    if row['dsm_iv_clinical_diagnosis'] == 1 and row['ever_tbi_w_loc'] == "Y":
        donor_df.loc[i, 'Treatment'] = 3
    elif row['dsm_iv_clinical_diagnosis'] == 1 and row['ever_tbi_w_loc'] == "N":
        donor_df.loc[i, 'Treatment'] = 2
    elif row['dsm_iv_clinical_diagnosis'] == 0 and row['ever_tbi_w_loc'] == "Y":
        donor_df.loc[i, 'Treatment'] = 1
    elif row['dsm_iv_clinical_diagnosis'] == 0 and row['ever_tbi_w_loc'] == "N":
        donor_df.loc[i, 'Treatment'] = 0

In [16]:
sample_df = pd.read_csv("tbi_data.csv")
region = "HIP" # values: ["FWM", "HIP", "PCx", "TCx"]
group = "FULL" # values: ["AD", "FULL"]
check = False # flag for whether to check for missing files. False assumes all files are available.

if group == "AD":
    data_df = donor_df[(donor_df["Treatment"] == 3) | (donor_df["Treatment"] == 2)]

data_treatments = donor_df[['name', 'Treatment']]
data_treatments["UID"] = data_treatments["name"].apply(lambda x: x.split(".")[2])
data_treatments.drop("name", axis=1, inplace=True)

sample_df["UID"] = sample_df["donor_name"].apply(lambda x: x.split(".")[2])
data_full = sample_df.merge(data_treatments, on=["UID"], how="left")
data_full = data_full.loc[(data_full["structure_acronym"] == region)]
data_full.dropna(inplace=True)

if checkformissing == True:
    filelist = checkformissing()
    data_full = data_full.loc[(data_full["structure_acronym"] == region) & (data_full["UID"].isin(filelist))]

# generate .csv file for R scripts
# data_full.to_csv("PCx_AD.csv") 

# generate treatment list for R scripts
string = "("
for i, row in data_full.iterrows():
    if row['structure_acronym'] == region:
        string+= str(int(row['Treatment']))
        string+= ","
string = string[:-1] + ")"
string

C:\Users\Vasilis\AppData\Local\Temp\ipykernel_17608\1207004099.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_treatments["UID"] = data_treatments["name"].apply(lambda x: x.split(".")[2])
C:\Users\Vasilis\AppData\Local\Temp\ipykernel_17608\1207004099.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_treatments.drop("name", axis=1, inplace=True)


'(3,0,3,2,1,2,1,0,1,0,0,2,3,3,0,3,2,3,2,1,0,3,2,1,2,1,2,1,0,3,2,3,0,0,3,2,1,0,3,2,3,2,3,0,1,0,3,2,1,2,1,2,1,0,3,0,1,0,3,2,1,2,2,1,2,2,2,3,0,1,2,3,0,1,0,1,2,1,0,1,2,3,2,1,0,3,2,3,2,1,2,1,0)'

In [48]:
# alternative AD classification

for i, row in data_df.iterrows():
    if row[['cerad', 'dsm_iv_clinical_diagnosis', 'nincds_arda_diagnosis', 'act_demented', 'braak']].sum() > 2:
        data_df.loc[i, 'AD'] = 1
    else:
        data_df.loc[i, 'AD'] = 0